Import Libraries

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

Launch Spark Context

In [2]:
import findspark
findspark.init()
print findspark.find()
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)

/usr/local/opt/apache-spark/libexec


Check child execution processes

In [3]:
import sys
rdd = sc.parallelize(xrange(10),10)
rdd.map(lambda x: sys.version).collect()

['2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x8

In [4]:
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
sqlsc=SQLContext(sc)

In [5]:
#colnames=['bounding_box','city','country','text','tid','timestamp','uid']
#pandas_df = pd.read_csv('./tweets.csv', delimiter=',', skiprows=1, names=colnames, usecols=[1,2,3,4,5,6,7])
#pandas_df = pd.read_csv('./tweets.csv', usecols=[3])
#pandas_df.head(5)
#alltweets_df = sqlsc.createDataFrame(pandas_df)
#alltweets_df.printSchema()
#alltweets_df.show(5)
#ustweets_df = alltweets_df[alltweets_df['country'] == "United States"]
#ustweets_df = alltweets_df.filter(alltweets_df['country'] == "United States")
#print "Count of All tweets =", alltweets_df.count()
#print "Count of US tweets:", ustweets_df.count()
#ustweets_df.show(5)
#pandas_df.country.head()

In [6]:
alltweets_rdd = sc.textFile('./tweets.csv')
alltweets_rdd = alltweets_rdd.map(lambda line: line.split(","))
print "Total number of tweets: ", alltweets_rdd.count()
alltweets_rdd.take(2)

Total number of tweets:  37519


[[u'0',
  u'"{u\'type\': u\'Polygon\'',
  u" u'coordinates': [[[-118.082615",
  u' 33.628991]',
  u' [-118.082615',
  u' 33.756093]',
  u' [-117.91485',
  u' 33.756093]',
  u' [-117.91485',
  u' 33.628991]]]}"',
  u'Huntington Beach',
  u'United States',
  u'In need of back rub',
  u'669375138903666692',
  u'1448426444160',
  u'1611125173'],
 [u'1',
  u'"{u\'type\': u\'Polygon\'',
  u" u'coordinates': [[[-82.87387",
  u' 38.688052]',
  u' [-82.87387',
  u' 38.76256]',
  u' [-82.811927',
  u' 38.76256]',
  u' [-82.811927',
  u' 38.688052]]]}"',
  u'Wheelersburg',
  u'United States',
  u"And I'm always tired but never of you",
  u'669375138958307328',
  u'1448426444173',
  u'3165522051']]

In [7]:
alltweets_rdd = sc.textFile('./tweets.csv')
alltweets_rdd = alltweets_rdd.map(lambda line: line.split(","))
header = alltweets_rdd.first()
alltweets_rdd = alltweets_rdd.filter(lambda line:line != header)
#alltweets_df = alltweets_rdd.map(lambda line: Row(bounding_box=line[1:10], city=line[10], country=line[11], 
#                              text=line[12:-3], tid=line[-3], timestamp=line[-2], uid=line[-1])).toDF()
alltweets_df0 = alltweets_rdd.map(lambda line: Row(bounding_box=line[1:10], city=line[10], country=line[11], 
                              text=line[12:-3], tid=line[-3], timestamp=line[-2], uid=line[-1]))
alltweets_df = sqlsc.createDataFrame(alltweets_df0)
alltweets_df.registerTempTable("alltweets_df")
alltweets_df.printSchema()

print type(alltweets_rdd)
print alltweets_rdd.count()
print type(alltweets_df)
# note alltweets_df is not and RDD, will not take RDD actions like count() or take()
# e.g the following wont work    
#print alltweets_df.count()

root
 |-- bounding_box: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tid: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- uid: string (nullable = true)

<class 'pyspark.rdd.PipelinedRDD'>
37518
<class 'pyspark.sql.dataframe.DataFrame'>


In [8]:
ustweets_df = alltweets_df.filter(alltweets_df['country'] == "United States")
print type(ustweets_df)
ustweets_df.first()

<class 'pyspark.sql.dataframe.DataFrame'>


Row(bounding_box=[u'"{u\'type\': u\'Polygon\'', u" u'coordinates': [[[-82.87387", u' 38.688052]', u' [-82.87387', u' 38.76256]', u' [-82.811927', u' 38.76256]', u' [-82.811927', u' 38.688052]]]}"'], city=u'Wheelersburg', country=u'United States', text=[u"And I'm always tired but never of you"], tid=u'669375138958307328', timestamp=u'1448426444173', uid=u'3165522051')

In [9]:
#tweets = sqlsc.sql("SELECT * FROM ustweets_df")
#print tweets.count()

####function to process tweet texts

In [10]:
def process_tweets(text):
    return text

In [11]:
processed_tweets = ustweets_df.map(lambda r: process_tweets(r.text))
print type(processed_tweets)

<class 'pyspark.rdd.PipelinedRDD'>


In [12]:
# The results of SQL queries are RDDs and support all the normal RDD operations.
#teenNames = teenagers.map(lambda p: "Name: " + p.name)
#for teenName in teenNames.collect():
#  print(teenName)

In [13]:
# On my Mac
#milliontweets = sqlsc.read.json("/Users/akinyi/Downloads/omtm.json")
#milliontweets = sqlsc.read.json("./omtm.json")
milliontweets = sqlsc.read.json("./result.json")

In [14]:
milliontweets.cache()

DataFrame[coords: string, country: string, lang: string, place: string, timestamp: string, ttext: string, tweetid: string, userid: bigint]

In [15]:
milliontweets.printSchema()

root
 |-- coords: string (nullable = true)
 |-- country: string (nullable = true)
 |-- lang: string (nullable = true)
 |-- place: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- ttext: string (nullable = true)
 |-- tweetid: string (nullable = true)
 |-- userid: long (nullable = true)



In [17]:
milliontweets.count()

14

In [18]:
milliontweets.show(5)

+--------------------+-------------+----+-----------------+-------------+--------------------+------------------+----------+
|              coords|      country|lang|            place|    timestamp|               ttext|           tweetid|    userid|
+--------------------+-------------+----+-----------------+-------------+--------------------+------------------+----------+
|47.61493799999999...|United States|  en|      Seattle, WA|1447266708839|After taking publ...|664510856407834624| 537328079|
|37.7050435,-122.1...|United States|  en|  San Leandro, CA|1447266710015|Thankful for all ...|664510861340340228|3896359752|
|37.7706565,-122.4...|United States|  en|San Francisco, CA|1447266716814|@Priz I've been w...|664510889857433600|  15532647|
|26.27026099999999...|United States|  en|Coral Springs, FL|1447266717060|@awolnation it's ...|664510890889342976|1064780006|
|39.7795625,-86.14...|United States|  en| Indianapolis, IN|1447266716818|@bobandtom VIP ca...|664510889874186240|  63448519|


In [39]:
import json
#line_generator = open("/Users/akinyi/Downloads/omtm.json")
#line_generator = open("./small.omtm.json").read()
for line in line_generator:
    line_object = json.loads(line)

    actor_id_string = line_object["source"]["text"]
    #actor_id = int( actor_id_string.split(":")[2] )
    #language_code = line_object["twitter_lang"]

ValueError: Expecting object: line 1 column 3 (char 2)

In [2]:
import json
with open("./result.json", "r") as fd:
    yearinfo = json.load(fd)

In [ ]:
with open("./omtm.json", "r") as fd:
    yearinfo = json.load(fd)

In [ ]:
tweetsdict={}
for y in yearinfo.keys():
    yearlist=yearinfo[y]
    yearlist2=[]
    for idict in yearlist:
        singers=idict['band_singer']
        for i,s in enumerate(singers):
            songs=idict['song']
            for j,so in enumerate(songs):#now inside each singer song combination
                nd={}
                nd['band_singer']=s
                nd['url']=idict['url'][i]
                nd['song']=so
                nd['songurl']=idict['songurl'][j]
                nd['ranking']=idict['ranking']
                yearlist2.append(nd)
    yeardict[y]=pd.DataFrame(yearlist2)#one for each year
yearspanel=pd.Panel.from_dict(yeardict, orient="minor")#stack dataframes into a panel
hierframe=yearspanel.to_frame() #flattening leads to a hierarchical index